# test SAX: Generic PDK

In [ ]:
import gdsfactory as gf
import matplotlib.pyplot as plt
import numpy as np
import sax
from gdsfactory.components import mzi

c = gf.Component()
mzi1_ = c << mzi(delta_length=10)
c.add_port("o1", port=mzi1_.ports["o1"])
c.add_port("o2", port=mzi1_.ports["o2"])
c.plot()

In [ ]:
def bend_euler(
    angle=90.0,
    p=0.5,
):
    return sax.reciprocal({("o1", "o2"): 1.0})


def mmi1x2():
    """Assumes a perfect 1x2 splitter."""
    return sax.reciprocal(
        {
            ("o1", "o2"): 0.5**0.5,
            ("o1", "o3"): 0.5**0.5,
        }
    )


def mmi2x2():
    S = {
        ("o1", "o3"): 0.5**0.5,
        ("o1", "o4"): 1j * 0.5**0.5,
        ("o2", "o3"): 1j * 0.5**0.5,
        ("o2", "o4"): 0.5**0.5,
    }
    return sax.reciprocal(S)


def straight(wl=1.55, length=10.0, width=0.5):
    S = {
        ("o1", "o2"): np.exp(2j * np.pi * 3.1 / wl * length),
    }
    return sax.reciprocal(S)


recnet = sax.RecursiveNetlist.model_validate(c.get_netlist(recursive=True))
# pprint(c.get_netlist(recursive=True))
# print('============')
print("Required models:", sax.get_required_circuit_models(recnet))
print("============")

models = {
    "straight": straight,
    "bend_euler": bend_euler,
    "mmi1x2": mmi1x2,
}
# print('Required models+models:', sax.get_required_circuit_models(recnet, models))
# print('============')

_mzi, info = sax.circuit(recnet, models, backend="default")
print("s-params:", _mzi())

# DemoPDK

In [ ]:
import matplotlib

matplotlib.use("macosx")

from gdsfactory.generic_tech import get_generic_pdk

genericPDK = get_generic_pdk()

LAYER = genericPDK.layers
layer_views = genericPDK.layer_views
layer_stack = genericPDK.layer_stack
cross_sections = genericPDK.cross_sections

%matplotlib inline
import glob
import importlib
import sys

import gdsfactory as gf
import sax

sys.path.append("/Users/vahid/Downloads/PhotonicsAI_Project")


def list_python_files(directory):
    """Lists all Python files in the given directory using glob."""
    # Use glob to list all .py files, exclude __init__.py
    return [
        f.split("/")[-1][:-3]
        for f in glob.glob(f"{directory}/*.py")
        if not f.endswith("__init__.py")
    ]


def import_modules(module_names):
    """Imports the specified function from each module in the given package and returns them in a dictionary."""
    functions = {}
    for module_name in module_names:
        full_module_name = f"PhotonicsAI.KnowledgeBase.DesignLibrary.{module_name}"
        module = importlib.import_module(full_module_name)
        func = getattr(module, module_name)
        globals()[module_name] = func
        functions[module_name] = func
    return functions


def import_models(module_names):
    """Imports the specified function from each module in the given package and returns them in a dictionary."""
    models_dict = {}
    for module_name in module_names:
        full_module_name = f"PhotonicsAI.KnowledgeBase.DesignLibrary.{module_name}"
        module = importlib.import_module(full_module_name)
        func = module.get_model
        models_dict.update(func())
        # globals()[module_name] = func()
    return models_dict


module_names = list_python_files("../../KnowledgeBase/DesignLibrary/")
cells = import_modules(module_names)
all_models = import_models(module_names)

DemoPDK = gf.Pdk(
    name="DemoPDK",
    layers=LAYER,
    cross_sections=cross_sections,
    cells=cells,
    layer_views=layer_views,
)
DemoPDK.activate()

DemoPDK = gf.Pdk(
    name="DemoPDK",
    layers=LAYER,
    cross_sections=cross_sections,
    cells=cells,
    layer_views=layer_views,
)
DemoPDK.activate()


c = gf.Component()
ref1 = c << DemoPDK.get_component("mzi1", delta_length=20)
c.plot()

In [ ]:
recnet = sax.RecursiveNetlist.model_validate(c.get_netlist(recursive=True))
# pprint(c.get_netlist(recursive=True))
print("============")
print("Required models:", sax.get_required_circuit_models(recnet))
print("============")

In [ ]:
netlist1 = """
instances:
  C1:
    component: mzi1
    info:
      ports: 2x2
    settings:
      delta_length: 10
      length: 10
placements:
  C1:
    x: 3.0
    y: 1.0
ports:
    o4: C1,o4
    o3: C1,o3
    o2: C1,o2
    o1: C1,o1
"""

netlist1 = """
instances:
  C1:
    component: wdm_mzi1x4
    info:
      dx: 642.813
      dy: 757.27
      ports: 1x4
    settings:
      dy: 120
  C2:
    component: mzi_1x1_pindiode_cband
    info:
      dx: 435.6
      dy: 462.2
      ports: 1x1
    settings:
      length: 320
  C3:
    component: mzi_1x1_pindiode_cband
    info:
      dx: 435.6
      dy: 462.2
      ports: 1x1
    settings:
      length: 320
  C4:
    component: mzi_1x1_pindiode_cband
    info:
      dx: 435.6
      dy: 462.2
      ports: 1x1
    settings:
      length: 320
  C5:
    component: mzi_1x1_pindiode_cband
    info:
      dx: 435.6
      dy: 462.2
      ports: 1x1
    settings:
      length: 320
placements:
  C1:
    x: 99.99650000000003
    y: 1334.569
  C2:
    x: 953.9080000000001
    y: 2340.139
  C3:
    x: 953.9080000000001
    y: 1670.694
  C4:
    x: 953.9080000000001
    y: 1001.236
  C5:
    x: 953.9080000000001
    y: 331.792
ports:
  o1: C1,o1
  o2: C2,o2
  o3: C3,o2
  o4: C4,o2
  o5: C5,o2
routes:
  optical:
    links:
      C1,o2: C2,o1
      C1,o3: C3,o1
      C1,o4: C4,o1
      C1,o5: C5,o1
"""

netlist1 = """
instances:
  C1:
    component: mzi_2x2_heater_tin_cband
    info:
      dx: 516.4000000000001
      dy: 295.07
      ports: 2x2
    settings:
      length: 320
  C2:
    component: _directional_coupler
    info:
      dx: 40.0
      dy: 4.5
      ports: 2x2
    settings:
      dx: 100
      dy: 100
      length: 125
placements:
  C1:
    x: 99.99399999999997
    y: 248.25
  C2:
    x: 827.514
    y: 248.25
ports:
  o1: C1,o2
  o2: C1,o1
  o3: C2,o3
  o4: C2,o4
routes:
  optical:
    links:
      C1,o3: C2,o2
      C1,o4: C2,o1
"""

# netlist1 = '''
# instances:
#   C1:
#     component: mzi_2x2_heater_tin_cband
# ports:
#   o1: C1,o2
#   o2: C1,o1
#   o3: C1,o3
#   o4: C1,o4
# '''


netlist1 = """
instances:
  C1:
    component: _mmi2x2
    info:
      dx: 25.5
      dy: 2.5
      ports: 2x2
    settings:
      cross_section: strip
      gap_mmi: 0.25
      length_mmi: 5.5
      length_taper: 10
      width_mmi: 2.5
      width_taper: 1
  C2:
    component: _mmi2x2
    info:
      dx: 25.5
      dy: 2.5
      ports: 2x2
    settings:
      cross_section: strip
      gap_mmi: 0.25
      length_mmi: 5.5
      length_taper: 10
      width_mmi: 2.5
      width_taper: 1
placements:
  C1:
    x: 99.75
    y: 102.083
  C2:
    x: 335.861
    y: 102.083
ports:
  o1: C1,o2
  o2: C1,o1
  o3: C2,o3
  o4: C2,o4
routes:
  optical:
    links:
      C1,o3: C2,o2
      C1,o4: C2,o1
"""

netlist1 = """
instances:
  C1:
    component: _mmi2x2
  C2:
    component: _mmi2x2
ports:
  o1: C1,o2
  o2: C1,o1
  o3: C2,o3
  o4: C2,o4
placements:
  C1:
    x: 0
    y: 0
  C2:
    x: 40
    y: 0
routes:
  optical:
    links:
      C1,o3: C2,o2
      C1,o4: C2,o1
"""

c = gf.read.from_yaml(netlist1)
c.plot()
# pprint(c.get_netlist())
# print('=====================')

gf_netlist_dict_recursive = c.get_netlist(recursive=True)
# recursive_netlist = sax.RecursiveNetlist.model_validate(gf_netlist_dict_recursive)
required_models = sax.get_required_circuit_models(gf_netlist_dict_recursive)
print("Required Models ==>", required_models)
# print('=====================')


def bend_euler(
    angle=90.0,
    p=0.5,
):
    return sax.reciprocal({("o1", "o2"): 1.0})


def mmi2x2():
    S = {
        ("o1", "o3"): 0.5**0.5,
        ("o1", "o4"): 1j * 0.5**0.5,
        ("o2", "o3"): 1j * 0.5**0.5,
        ("o2", "o4"): 0.5**0.5,
    }
    return sax.reciprocal(S)


def straight(wl=1.55, length=10.0, width=0.5):
    S = {("o1", "o2"): np.exp(2j * np.pi * 3.1 / wl * length)}
    return sax.reciprocal(S)


ana_models = {
    "straight": straight,
    "bend_euler": bend_euler,
    "_mmi2x2": all_models["_mmi2x2"],
}

# all_models['_mmi1x2']

_mzi, info = sax.circuit(gf_netlist_dict_recursive, ana_models, backend="default")
print("s-params:", _mzi())

wl = np.linspace(1.5, 1.6, 100)
result = _mzi(wl=wl)
plt.figure()
plt.plot(np.abs(result[("o1", "o3")]))
plt.plot(np.abs(result[("o1", "o4")]))

In [ ]:
def mmi1x2(
    width=0.5,
    width_taper=1.0,
    length_taper=10.0,
    length_mmi=5.5,
    width_mmi=2.5,
    gap_mmi=0.25,
):
    return sax.reciprocal(
        {
            ("o1", "o2"): 0.45**0.5,
            ("o1", "o3"): 0.45**0.5,
        }
    )


print(mmi1x2())

# print(circuit_models['_mmi1x2'])

In [ ]:
m = all_models["_mmi2x2"]

wl = np.linspace(1.4, 1.6, 100)
result = m(wl=wl)

S = np.abs(result[("o1", "o3")])
Sa = np.angle(result[("o1", "o3")])

plt.figure(figsize=(15, 3))
plt.suptitle("_mmi2x2")
plt.subplot(121)
plt.plot(wl, np.abs(result[("o1", "o3")]))
plt.subplot(122)
plt.plot(wl, np.angle(result[("o1", "o3")]))

print(type(np.angle(S)))

In [ ]:
result[("o1", "o2")]